In [1]:
import numpy as np
indices = np.random.choice(10, 5, replace=True)
arr = np.array([1, 2, 3, 4, 5, 6, 7, 8, 9, 10])
print(arr[indices])
print(arr[-11:])
reward_buffer = [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, -1]
size = 7
game_start_index = 0
reward = reward_buffer[size - 1]
updated_rewards = np.empty((size - game_start_index,), int)
updated_rewards[::2] = 1 * reward
updated_rewards[1::2] = -1 * reward
updated_rewards = np.flip(updated_rewards)
print(updated_rewards)
reward_buffer[game_start_index:size] = updated_rewards
print(reward_buffer)
game_start_index = 7
size = 15
print(len(reward_buffer))
reward = reward_buffer[size - 1]
updated_rewards = np.empty((size - game_start_index,), int)
updated_rewards[::2] = 1 * reward
updated_rewards[1::2] = -1 * reward
updated_rewards = np.flip(updated_rewards)
print(updated_rewards)
reward_buffer[game_start_index:size] = updated_rewards
print(reward_buffer)
print(len(reward_buffer))
overflow_reward = 1
overflow_count = 1

[9 3 4 2 3]
[ 1  2  3  4  5  6  7  8  9 10]
[ 1 -1  1 -1  1 -1  1]
[1, -1, 1, -1, 1, -1, 1, 0, 0, 0, 0, 0, 0, 0, -1]
15
[ 1 -1  1 -1  1 -1  1 -1]
[1, -1, 1, -1, 1, -1, 1, 1, -1, 1, -1, 1, -1, 1, -1]
15


In [2]:
# reward_buffer = np.array([1, -1, 1, -1, 1, -1, 0])
# reward_buffer[6] = [1, 1, 1, 1]
# print(reward_buffer)

In [3]:
# from alphazero_agent import AlphaZeroAgent
import gymnasium as gym 
import tensorflow as tf
import numpy as np
# import gym_envs

class ClipReward(gym.RewardWrapper):
    def __init__(self, env, min_reward, max_reward):
        super().__init__(env)
        self.min_reward = min_reward
        self.max_reward = max_reward
        self.reward_range = (min_reward, max_reward)

    def reward(self, reward):
        return np.clip(reward, self.min_reward, self.max_reward)


# env = ClipReward(gym.wrappers.AtariPreprocessing(gym.make("MsPacmanNoFrameskip-v4", render_mode="rgb_array"), terminal_on_life_loss=True), -1, 1) # as recommended by the original paper, should already include max pooling
env = TicTacToeEnv()
# env = gym.make("MsPacmanNoFrameskip-v4", render_mode="rgb_array")
env = gym.wrappers.FrameStack(env, 4)

config = {
        'activation': 'relu',
        'kernel_initializer': 'orthogonal',
        'optimizer_function': tf.keras.optimizers.legacy.Adam,
        'learning_rate': 0.2,
        'adam_epsilon': 1e-7,
        'clipnorm': 0.5,
        # NORMALIZATION?
        # REWARD CLIPPING
        'num_epochs': 30,
        'num_filters': 32,
        'kernel_size': 3,
        'stride': 1,
        'num_res_blocks': 5,
        'critic_conv_filters': 32,
        'critic_conv_layers': 1,
        'critic_dense_size': 32,
        'critic_dense_layers': 1,
        'actor_conv_filters': 32,
        'actor_conv_layers': 1,
        'actor_dense_size': 32,
        'actor_dense_layers': 1,
        'memory_size': 180,
        'max_game_length': 9,
        'batch_size': 9,
        'dirichlet_alpha': 0.3,
        'dirichlet_epsilon': 0.25,
        'c_puct': 1,
        'monte_carlo_simulations': 18,
        'two_player': True,
        'weight_decay': 1e-4,
    }    



/Users/jonathanlamontange-kratz/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


NameError: name 'TicTacToeEnv' is not defined

In [ ]:
agent = AlphaZeroAgent(env, config=config)
agent.train()

In [ ]:
array = np.zeros((3, 9))
array[0] = [1, 2, 3, 4, 5, 6, 7, 8, 9]
print(array)

In [ ]:
import gym_envs
import gymnasium as gym
env = gym.make('gym_envs/TicTacToe-v0')

state, info = env.reset()
print(state)
print("Turn: ", state[2][0][0])
print("Legal moves: ", info['legal_moves'])
env.render()
state, reward, terminated, truncated, info = env.step(0)
print(state)
print("Turn: ", state[2][0][0])
print("Legal moves: ", info['legal_moves'])
print("Terminated:", terminated)
env.render()
state, reward, terminated, truncated, info = env.step(4)
print(state)
print("Turn: ", state[2][0][0])
print("Legal moves: ", info['legal_moves'])
print("Terminated:", terminated)
env.render()
state, reward, terminated, truncated, info = env.step(3)
print(state)
print("Turn: ", state[2][0][0])
print("Legal moves: ", info['legal_moves'])
print("Terminated:", terminated)
env.render()
state, reward, terminated, truncated, info = env.step(6)
print(state)
print("Turn: ", state[2][0][0])
print("Legal moves: ", info['legal_moves'])
print("Terminated:", terminated)
env.render()
state, reward, terminated, truncated, info = env.step(2)
print(state)
print("Turn: ", state[2][0][0])
print("Legal moves: ", info['legal_moves'])
print("Terminated:", terminated)
env.render()
state, reward, terminated, truncated, info = env.step(1)
print(state)
print("Turn: ", state[2][0][0])
print("Legal moves: ", info['legal_moves'])
print("Terminated:", terminated)
env.render()
state, reward, terminated, truncated, info = env.step(7)
print(state)
print("Turn: ", state[2][0][0])
print("Legal moves: ", info['legal_moves'])
print("Terminated:", terminated)
state, reward, terminated, truncated, info = env.step(8)
print(state)
print("Turn: ", state[2][0][0])
print("Legal moves: ", info['legal_moves'])
print("Terminated:", terminated)
state, reward, terminated, truncated, info = env.step(5)
print(state)
print("Turn: ", state[2][0][0])
print("Legal moves: ", info['legal_moves'])
print("Terminated:", terminated)
print("Truncated:", truncated)
env.render()


env.reset()
state, reward, terminated, truncated, info = env.step(0)
print(state)
print("Turn: ", state[2][0][0])
print("Legal moves: ", info['legal_moves'])
print("Terminated:", terminated)
state, reward, terminated, truncated, info = env.step(3)
print(state)
print("Turn: ", state[2][0][0])
print("Legal moves: ", info['legal_moves'])
print("Terminated:", terminated)
state, reward, terminated, truncated, info = env.step(7)
print(state)
print("Turn: ", state[2][0][0])
print("Legal moves: ", info['legal_moves'])
print("Terminated:", terminated)
state, reward, terminated, truncated, info = env.step(4)
print(state)
print("Turn: ", state[2][0][0])
print("Legal moves: ", info['legal_moves'])
print("Terminated:", terminated)
state, reward, terminated, truncated, info = env.step(2)
print(state)
print("Turn: ", state[2][0][0])
print("Legal moves: ", info['legal_moves'])
print("Terminated:", terminated)
state, reward, terminated, truncated, info = env.step(6)
print(state)
print("Turn: ", state[2][0][0])
print("Legal moves: ", info['legal_moves'])
print("Terminated:", terminated)
state, reward, terminated, truncated, info = env.step(1)
print(state)
print("Turn: ", state[2][0][0])
print("Legal moves: ", info['legal_moves'])
print("Terminated:", terminated)
print("Truncated:", truncated)
print("Reward:", reward)



In [ ]:
# import tensorflow as tf
# regularizer = tf.keras.regularizers.L2(2)
# print(regularizer(agent.model.trainable_variables))

In [ ]:
import numpy as np
import tensorflow as tf
arr = [1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0]

print(arr/np.sum(arr))
# print(tf.nn.softmax(arr))

In [2]:
from alphazero_agent import AlphaZeroAgent
# from alphazero_agent import AlphaZeroAgent
import gymnasium as gym
import tensorflow as tf
import numpy as np
import gym_envs

class ClipReward(gym.RewardWrapper):
    def __init__(self, env, min_reward, max_reward):
        super().__init__(env)
        self.min_reward = min_reward
        self.max_reward = max_reward
        self.reward_range = (min_reward, max_reward)

    def reward(self, reward):
        return np.clip(reward, self.min_reward, self.max_reward)


# env = ClipReward(gym.wrappers.AtariPreprocessing(gym.make("MsPacmanNoFrameskip-v4", render_mode="rgb_array"), terminal_on_life_loss=True), -1, 1) # as recommended by the original paper, should already include max pooling
# env = TicTacToeEnv(render_mode="rgb_array")
# env = gym.make("MsPacmanNoFrameskip-v4", render_mode="rgb_array")
# env = gym.wrappers.FrameStack(env, 4)
env = gym.make('gym_envs/TicTacToe-v0', render_mode="rgb_array")

config = {
        'activation': 'relu',
        'kernel_initializer': 'orthogonal',
        'optimizer': tf.keras.optimizers.legacy.Adam,
        'learning_rate': 0.1,
        'adam_epsilon': 3.25e-6,
        'clipnorm': None,
        # NORMALIZATION?
        # REWARD CLIPPING
        'training_steps': 100,
        'num_filters': 256,
        'kernel_size': 3,
        'stride': 1,
        'num_res_blocks': 20,
        'critic_conv_filters': 32, # 1
        'critic_conv_layers': 1,
        'critic_dense_size': 256,
        'critic_dense_layers': 1,
        'actor_conv_filters': 32, # 
        'actor_conv_layers': 1,
        'actor_dense_size': 0,
        'actor_dense_layers': 0,
        'replay_buffer_size': 3200, # IN GAMES
        'replay_batch_size': 300, # IN MOVES
        'root_dirichlet_alpha': 1.0, # 2 in theory?
        'root_exploration_fraction': 0.25,
        'pb_c_base': 2000,
        'pb_c_init': 4,
        'num_simulations': 800,
        # 'two_player': True,
        'weight_decay': 0.001, #0.5
        'num_sampling_moves': 5,
        'initial_temperature': 1,
        'exploitation_temperature': 0.05,
        'value_loss_factor': 0.1,
        'games_per_generation': 40,
    }

agent = AlphaZeroAgent(env, config=config, name="alphazero")
agent.train()

Training Step  1


/Users/jonathanlamontange-kratz/Library/Python/3.9/lib/python/site-packages/keras/src/initializers/initializers.py:120: UserWarning: The initializer Orthogonal is unseeded and being called multiple times, which will return identical values each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initializer instance more than once.
  warnings.warn(


Predicted Policy  [0.11111111 0.11111111 0.11111111 0.11111111 0.11111111 0.11111111
 0.11111111 0.11111111 0.11111111]
Predicted Value  [0.]
Target Policy [0.14       0.0875     0.1275     0.08875    0.07875    0.10875
 0.11875    0.11875    0.13124999]
Predicted Policy  [0.11386906 0.06492809 0.12867638 0.15030572 0.08149773 0.13044403
 0.12399183 0.11802193 0.08826522]
Predicted Value  [0.09315615]
